#### <font color = 'green'> TIME SERIES ANALYSIS AND MODELING AND INDUSTRIALIZATION FOR FORECASTING THE ENERGY CONSUMPTION IN THE INDUSTRY

###### Import necessary libaries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import pickle
import time
from datetime import datetime

###### Define the path to the folders containing the preprocessing and model artifacts

In [2]:
os.getcwd()

'C:\\Users\\Olanrewaju Adegoke\\Desktop\\TechTern\\mywork\\Time_Series_Analysis_Projects\\Energy_Consumption_Forecast_Project\\notebooks'

In [3]:
data_path = '../data'
model_path = '../models'
note_path = '../notebooks'

###### Function that load the model and preprocessing artifacts

In [4]:
def load_engineered_feature_future_artifact():
    os.chdir(model_path)
    with open('engineered_feature_forecast_model.pkl', 'rb') as file:
        features_forecast = pickle.load(file)
    return features_forecast

In [5]:
def load_energy_model_for_future():
    os.chdir(model_path)
    with open('total_energy_model.pkl', 'rb') as file:
        energy_model = pickle.load(file)
    return energy_model

###### Instantiate all the instances of the artifacts

In [6]:
os.chdir(model_path)
features_forecast = load_engineered_feature_future_artifact()
os.chdir(note_path)
hourly_lag = features_forecast['1h_lag']
eight_hour_lag = features_forecast['8h_lag']
twenty_four_hour_lag = features_forecast['24h_lag']
six_day_lag = features_forecast['6d_lag']
mv_avg_2hr_mean = features_forecast['mv_avg_2hr_mean']
mv_avg_2hr_std = features_forecast['mv_avg_2hr_std']
mv_avg_24hr_mean = features_forecast['mv_avg_24hr_mean']
mv_avg_24hr_std = features_forecast['mv_avg_24hr_std']

In [8]:
os.chdir(model_path)
energy_model = load_energy_model_for_future()
os.chdir(note_path)
total_energy_model = energy_model['total_energy_forecast_model']

###### The prediction and inference pipeline

In [9]:
hourly_lag
eight_hour_lag
twenty_four_hour_lag
six_day_lag
mv_avg_2hr_mean
mv_avg_2hr_std
mv_avg_24hr_mean
mv_avg_24hr_std

total_energy_model

def future_total_energy_prediction(start_date, num_of_hours):
    cols = ['1h_lag', '8h_lag', '24h_lag', '6d_lag', 'mv_avg_2hr_mean',
       'mv_avg_2hr_std', 'mv_avg_24hr_mean', 'mv_avg_24hr_std']

    start_date = pd.to_datetime(start_date)
    date_time_sequence = pd.date_range(start_date, periods=num_of_hours, freq='h')
    date_time_df = pd.DataFrame(date_time_sequence, columns=['date_time'])
    date_time = date_time_df.set_index('date_time')
    date_time['hour'] = date_time.index.hour
    date_time['day'] = date_time.index.dayofweek
    date_time['month'] = date_time.index.month
    date_time['year'] = date_time.index.year
    
    for col in cols:
        
        if col == '1h_lag':
            pred_1hr_lag = hourly_lag.predict(date_time)
            pred_1hr_lag_df = pd.DataFrame(pred_1hr_lag, columns=[col], index=date_time.index)
            
        if col == '8h_lag':
            
            pred_8hr_lag = eight_hour_lag.predict(date_time)
            pred_8hr_lag_df = pd.DataFrame(pred_8hr_lag, columns=[col], index=date_time.index)

        if col == '24h_lag':
            
            pred_24hr_lag = twenty_four_hour_lag.predict(date_time)
            pred_24hr_lag_df = pd.DataFrame(pred_24hr_lag, columns=[col], index=date_time.index)

        if col == '6d_lag':
            
            pred_6d_lag = six_day_lag.predict(date_time)
            pred_6d_lag_df = pd.DataFrame(pred_6d_lag, columns=[col], index=date_time.index)

        if col == 'mv_avg_2hr_mean':
            
            pred_mv_avg_2hr_mean = mv_avg_2hr_mean.predict(date_time)
            pred_mv_avg_2hr_mean_df = pd.DataFrame(pred_mv_avg_2hr_mean, columns=[col], index=date_time.index)

        if col == 'mv_avg_2hr_std':
            
            pred_mv_avg_2hr_std = mv_avg_2hr_std.predict(date_time)
            pred_mv_avg_2hr_std_df = pd.DataFrame(pred_mv_avg_2hr_std, columns=[col], index=date_time.index)

        if col == 'mv_avg_24hr_mean':
            
            pred_mv_avg_24hr_mean = mv_avg_24hr_mean.predict(date_time)
            pred_mv_avg_24hr_mean_df = pd.DataFrame(pred_mv_avg_24hr_mean, columns=[col], index=date_time.index)

        if col == 'mv_avg_24hr_std':

            pred_mv_avg_24hr_std = mv_avg_24hr_std.predict(date_time)
            pred_mv_avg_24hr_std_df = pd.DataFrame(pred_mv_avg_24hr_std, columns=[col], index=date_time.index)
    
    futuristic_features = pd.concat([pred_1hr_lag_df, pred_8hr_lag_df, pred_24hr_lag_df, pred_6d_lag_df, pred_mv_avg_2hr_mean_df,
                            pred_mv_avg_2hr_std_df, pred_mv_avg_24hr_mean_df, pred_mv_avg_24hr_std_df, date_time], axis=1)

    # Prediction
    total_energy_predicted = total_energy_model.predict(futuristic_features)
    total_energy_df = pd.DataFrame(total_energy_predicted, columns=[f'future_{num_of_hours}_hours_of_total_energy'], index=date_time.index)
       
    return total_energy_df

###### Install gradio GUI interface

###### Launch the gradio interface for industrializing the model

In [10]:
import gradio as gr

sample_data = [['2026-02-13 06:59:59', 300], ['2030-01-01 00:00:00', 4032]]

inputs=[
        gr.Textbox(label='start_date'),
        gr.Number(label='num_of_hours')
    ]

outputs = [gr.Dataframe(row_count = (1, "dynamic"), col_count=(1, "fixed"), label="Energy Consumption Prediction in Watts", headers=['energy_consumed'])]

title='Time Series Model for Energy Consumption Forecast for any number of hours into the future by Olanrewaju Adegoke'
description='You want to estimate the energy consumption for the next 1000 hours, provide the two information and have your energy consumption estimated.'

interface = gr.Interface(fn = future_total_energy_prediction, inputs = inputs, outputs = outputs, title=title, description=description, examples=sample_data)

interface.launch(share=True)

C:\Users\Olanrewaju Adegoke\Desktop\TechTern\mywork\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7863


C:\Users\Olanrewaju Adegoke\Desktop\TechTern\mywork\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.37.2, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://ce086cad8917812b50.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
